### Generative AI Basics (I)
Making an OpenAI API call

In [14]:
# Importing the OpenAI module

from openai import OpenAI
from toolz import second

# Initialise the OpenAI client
client = OpenAI()
gpt_model="gpt-4o" # Specify the GPT model to use

# Define the conversation messages
messages = [
    {"role": "system", "content": "You are an expert data scientist."}, # system message to set the assistant's behaviour
    {"role": "user", "content": "Write a Python script that trains a neural network on simulated data"} #User's question
]


In [16]:
# Send the messages to the Open AI API and receive a response

response = client.chat.completions.create(
    model=gpt_model,
    messages=messages,
    temperature=0.1 # Set the temperature for response creativity (lower is more deterministic)
)

# Extract the assistant's reply
assistant_reply = response.choices[0].message.content

print("Assistant's Reply:")
print(assistant_reply)


Assistant's Reply:
Certainly! Below is a Python script that demonstrates how to train a simple neural network using simulated data. We'll use the popular deep learning library, TensorFlow with Keras, to build and train the model. The simulated data will be generated using NumPy.

This example will create a simple regression problem where the goal is to predict a continuous target variable based on a set of input features.

```python
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Generate simulated data
def generate_data(num_samples=1000, num_features=10):
    X = np.random.rand(num_samples, num_features)
    # Create a target variable with some noise
    y = np.dot(X, np.random.rand(num_features)) + np.random.normal(0, 

### PromptTemplate Usage
The PromptTemplate provides a structured way to create dynamic and reusable templates. It:
1. Allows you to easily parameterize and substitute variables in a prompt.
2. Makes code cleaner and more maintainable when working with repetitive or complex prompts.
3. Ensures consistency in prompt design, especially in collaborative or large-scale projects.
4. Simplifies experimentation with different input variables, enabling better fine-tuning of the model's output.

In [19]:
from langchain import PromptTemplate

template = """
You are an expert data scientist with an expertise in building deep learning models.
Explain the concept of {concept} in a couple of lines
"""

prompt = PromptTemplate(
    input_variable=["concept"],
    template=template,
)

In [21]:
concept_to_explain = "neural networks"
formatted_prompt = prompt.format(concept=concept_to_explain)
print(formatted_prompt)


You are an expert data scientist with an expertise in building deep learning models.
Explain the concept of neural networks in a couple of lines



In [22]:
messages = [
    {"role": "system", "content": "You are an expert data scientist."},
    {"role": "user", "content": formatted_prompt}
]


response = client.chat.completions.create(
    model=gpt_model,
    messages=messages,
    temperature=0.1 # Set the temperature for response creativity (lower is more deterministic)
)

# Extract the assistant's reply
assistant_reply = response.choices[0].message.content

print("Assistant's Reply:")
print(assistant_reply)


Assistant's Reply:
Neural networks are computational models inspired by the human brain, consisting of interconnected layers of nodes (neurons) that process and transform input data to learn patterns and make predictions. They are the foundation of deep learning, enabling machines to perform complex tasks such as image and speech recognition by adjusting the weights of connections through training.


# Chaining
Example to chain the output of the first API call as the input of the second API call. 
It demonstrates how to dynamically build upon previous responses, enabling multi-step interactions.

In [24]:
# Step 1: Generate the first prompt and get the response
concept_to_explain = "neural networks"
formatted_prompt = prompt.format(concept=concept_to_explain)

# Define the conversation messages
messages = [
    {"role": "user", "content": formatted_prompt},
]

# Send the messages to the OpenAI API and receive a response
response = client.chat.completions.create(
    model=gpt_model,
    messages=messages,
    temperature=0.1 # Set the temperature for response creativity (lower is more deterministic)
)

# Extract the assistant's first reply
first_reply = response.choices[0].message.content

#Step 2: Use the first output as an input for the second query
follow_up_prompt = f"Based on your explanation: {first_reply}, can you give a practical example of its application?"

follow_up_messages = [
    {"role": "user", "content": follow_up_prompt},
]  

#Send the follow-up messages to the OpenAI API and receive a response
follow_up_response = client.chat.completions.create(
    model=gpt_model,
    messages=follow_up_messages,
    temperature=0.1 # Set the temperature for response creativity (lower is more deterministic)
)

#Extract the asssistant's second reply
second_reply = follow_up_response.choices[0].message.content

print("First Reply:")
print(first_reply)
print("\nSecond Reply:")
print(second_reply)



First Reply:
Neural networks are computational models inspired by the human brain, consisting of interconnected layers of nodes (neurons) that process and transform input data to learn patterns and make predictions. They are the foundation of deep learning, enabling machines to perform complex tasks such as image and speech recognition by adjusting the weights of connections through training.

Second Reply:
Certainly! One practical example of neural networks in action is in the field of medical imaging, specifically in the detection and diagnosis of diseases from medical scans, such as X-rays, MRIs, or CT scans.

### Application: Medical Image Analysis

#### Problem:
Radiologists often need to analyze medical images to detect abnormalities such as tumors, fractures, or other signs of disease. This process can be time-consuming and is subject to human error.

#### Solution:
Neural networks, particularly convolutional neural networks (CNNs), can be trained to automatically analyze medica

### Example of CLD (No Chaining)

In [32]:
system_message = r"""
You are a systems engineer tasked with analyzing text to identify variables and their causal relationships for constructing a Causal Loop Diagram (CLD). Follow these steps:

1. **Identify Primary Variables**  
   - First, read through the original input text carefully.  
   - Extract all key entities or components (e.g., “salesmen,” “orders,” “product delivery,” “revenue,” etc.).  
   - If certain variables overlap or share commonalities, group them thematically (e.g., “sales workforce,” “budget,” “capacity,” etc.).  

2. **Break Down Sub-Factors**  
   - Within each primary variable, look for nuanced factors that might influence or be influenced by others (e.g., “delivery delay,” “acquisition delay,” “order backlog”).  

3. **Map Causal Relationships**  
   - For each pair of related variables, specify the relationship in the following format:  
     \[
       [Variable A] --> ([+/-]) [Variable B]
     \]  
     Use **(+)** if an increase in Variable A leads to an increase in Variable B. Use **(-)** if an increase in Variable A leads to a decrease in Variable B.  
   - Provide your **Reasoning** next: Explain how you determined the relationship. If something in the original text supports or illustrates this cause-effect linkage, reference it.  
   - Under **Relevant Text**, include the **exact sentence(s) or phrase(s)** from the original input text that justifies your relationship. Enclose these quotes clearly (e.g., in quotation marks).

4. **Add Feedback Loops**  
   - Identify any cyclical cause-effect paths (e.g., “salesmen → revenue → budget → more salesmen”).  
   - Label loops as either *Reinforcing (R)* or *Balancing (B)*, depending on whether they amplify or stabilize the system.  
   - Include a short note on how the loop works overall, mentioning how the sign of relationships causes growth (reinforcing) or equilibrium (balancing).  

5. **Account for Delays**  
   - If the text mentions time delays in processes (e.g., “acquisition delay,” “delivery delay”), highlight them.  
   - Explain how delays might shift or weaken the speed of the feedback loop’s response.  

6. **Suggest Interventions**  
   - Based on the relationships identified, propose one or more policy or managerial interventions that could alter the system’s outcome (e.g., “shorten acquisition delay,” “increase capacity earlier”).  

7. **Visualize: Generate a Causal Loop Diagram**  
   - Finally, draw a rough CLD or outline how the variables connect in a diagram, showing loops, signs (+/-), and any delays.  
   - You can use text-based or ASCII-based visualization if needed.

---

**Output Format**

For each relationship, follow this exact template:

1. **[Variable A] --> ([+/-]) [Variable B]**  
   **Reasoning:** [Explanation of how/why the relationship is positive or negative, referencing delays and any loops if applicable.]  
   **Relevant Text:** [Exact quote(s) from the original text that substantiate this relationship.]

Proceed until all relationships are mapped. Once done, provide:

- A consolidated **Feedback Loops** section, listing and labeling each loop (e.g., Loop R1, Loop B1) and a brief explanation of how it reinforces or balances the system.  
- A brief note on **Delays**, describing their impact on the system.  
- **Suggestions** or recommended interventions.  
- (Optional) A textual or graphical representation of the **Causal Loop Diagram** (CLD).

---

### Example of How to Apply This Prompt

Example 1:
**Input Excerpt**  
```
"More chickens lay more eggs, which hatch and add to the chicken population.
The more chickens, the more road crossing they will attempt. If there is any
traffic, more road crossings will lead to fewer chickens."
```

**Output**  
1. chickens --> (+) eggs  
   Reasoning: Increased chickens lead to more eggs being laid.  
   Relevant Text: "More chickens lay more eggs..."

2. eggs --> (+) chickens  
   Reasoning: Eggs hatch into chickens, thus increasing the chicken population.  
   Relevant Text: "...which hatch and add to the chicken population."

3. road crossing --> (-) chickens  
   Reasoning: More chickens crossing roads, combined with traffic, can lead to a decreased chicken population.  
   Relevant Text: "If there is any traffic, more road crossings will lead to fewer chickens."

Example 2:

**Input Excerpt**  
```
"Here is a situation in which salesmen book orders followed by
product delivery which generates revenue which produces the sales
budget which permits hiring still more salesmen. In short, salesmen
produce revenue to pay for the further expansion of the sales
effort. Orders booked increase the order backlog which increases the
delivery delay which makes the product less attractive and reduces
the order rate. The ordering of new production capacity is a
function of delivery delay only. Rising order backlog, as indicated
by delivery delay, is taken as an indication of inadequate capacity,
and orders for more capacity are placed. These orders, after an
acquisition delay, add to the production capacity. As the delivery
delay rises, production capacity is raised to bring down the
delivery delay."
```

**Output**  
1. Salesmen --> (+) Orders booked
Reasoning: Salesmen are responsible for booking orders which lead to
revenue generation.
Relevant Text: Here is a situation in which salesmen book orders
followed by product delivery which generates revenue which produces the
sales budget which permits hiring still more salesmen.
2. Revenue --> (+) Sales budget
Reasoning: Revenue generated from orders booked contributes to the
sales budget.
Relevant Text: Here is a situation in which salesmen book orders
followed by product delivery which generates revenue which produces the
sales budget which permits hiring still more salesmen.
3. Sales budget --> (+) Salesmen
Reasoning: The sales budget allows for the hiring of more salesmen,
expanding the sales effort.
Relevant Text: Here is a situation in which salesmen book orders
followed by product delivery which generates revenue which produces the
sales budget which permits hiring still more salesmen.
4. Orders booked --> (+) Order backlog
Reasoning: Orders booked contribute to the order backlog.
Relevant Text: Orders booked increase the order backlog which increases
the delivery delay which makes the product less attractive and reduces
the order rate.
5. Order backlog --> (+) Delivery delay
Reasoning: An increase in order backlog leads to longer delivery
delays.
Relevant Text: As the delivery delay rises, production capacity is
raised to bring down the delivery delay.
6. Delivery delay --> (-) Product attractiveness
Reasoning: Longer delivery delays make the product less attractive to
customers.
Relevant Text: Orders booked increase the order backlog which increases
the delivery delay which makes the product less attractive and reduces
the order rate.
7. Product attractiveness --> (-) Order rate
Reasoning: Reduced product attractiveness leads to a decrease in the
order rate.
Relevant Text: Orders booked increase the order backlog which increases
the delivery delay which makes the product less attractive and reduces
the order rate.
8. Delivery delay --> (+) Capacity orders
Reasoning: The decision to order new production capacity is based on
the current delivery delay.
Relevant Text: The ordering of new production capacity is a function of
delivery delay only.
9. Delivery delay --> (+) Capacity orders
Reasoning: Delivery delay is used as an indicator of inadequate
capacity, prompting orders for more capacity.
Relevant Text: Rising order backlog, as indicated by delivery delay, is
taken as an indication of inadequate capacity, and orders for more
capacity are placed.
10. Capacity orders --> (+) Production capacity
Reasoning: After an acquisition delay, the ordered capacity adds to the
existing production capacity.
Relevant Text: These orders, after an acquisition delay, add to the
production capacity.
11. Delivery delay --> (+) Production capacity
Reasoning: As delivery delay increases, production capacity is raised
to reduce the delivery delay.
Relevant Text: As the delivery delay rises, production capacity is
raised to bring down the delivery delay.
12. Production capacity --> (-) Delivery delay
Reasoning: Increased production capacity should reduce the delivery
delay.
Relevant Text: As the delivery delay rises, production capacity is
raised to bring down the delivery delay.
13. Order rate --> (-) Orders booked
Reasoning: Decreased order rate leads to fewer orders booked.
Relevant Text: Orders booked increase the order backlog which increases
the delivery delay which makes the product less attractive and reduces
the order rate.
14. Orders booked --> (-) Revenue
Reasoning: Fewer orders booked would lead to a decrease in revenue.
Relevant Text: Here is a situation in which salesmen book orders
followed by product delivery which generates revenue which produces the
sales budget which permits hiring still more salesmen.
"""

In [35]:
# Define the conversation messages
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": """
    Patient Information

    Patient Name: Mr. Smith
    Patient ID: 789456
    Date of Birth: 06/25/1965
    Date of Examination: 03/15/2024

MRI Examination Details

    MRI Technique: Multi-parametric MRI with T2-weighted images, diffusion-weighted imaging (DWI), and dynamic contrast-enhanced (DCE) MRI.
    Prostate Volume: 45 cc
    PSA Level: 5.8 ng/mL

MRI Findings

    Lesion 1: Located in the left peripheral zone, mid gland. It shows significant restricted diffusion with a corresponding low ADC value and focal early enhancement on DCE MRI. PI-RADS Score: 5.
    Lesion 2: Located in the right peripheral zone, apex. It has mildly restricted diffusion and no significant enhancement on DCE MRI. PI-RADS Score: 3.
    Lesion 3: Located in the anterior fibromuscular stroma, showing moderate restricted diffusion and no early enhancement. PI-RADS Score: 3."""},
]


In [38]:
# Send the messages to the OpenAI API and receive a response
response = client.chat.completions.create(
    model="chatgpt-4o-latest",
    messages=messages,
    temperature=0.1 # Set the temperature for response creativity (lower is more deterministic)
)

# Extract the assistant's reply
print(response.choices[0].message.content)

### Causal Loop Diagram (CLD) Analysis for MRI Findings and Prostate Health

---

#### **Step 1: Identify Primary Variables**
From the text, the following primary variables are identified:
1. **Prostate Volume**  
2. **PSA Level**  
3. **Lesion Characteristics**  
   - Lesion 1: Left peripheral zone, mid gland (PI-RADS 5)  
   - Lesion 2: Right peripheral zone, apex (PI-RADS 3)  
   - Lesion 3: Anterior fibromuscular stroma (PI-RADS 3)  
4. **MRI Parameters**  
   - Restricted diffusion (ADC value)  
   - Dynamic contrast enhancement (DCE MRI)  
5. **PI-RADS Score**  
6. **Cancer Risk**  

---

#### **Step 2: Break Down Sub-Factors**
Each primary variable has sub-factors that influence or are influenced by others:
- **Prostate Volume**: May correlate with PSA levels and lesion development.  
- **PSA Level**: Elevated PSA levels are often associated with prostate abnormalities, including cancer.  
- **Lesion Characteristics**: Include location, diffusion restriction, ADC value, and enha

### Example CLD with Chaining

In [53]:
system_message_1 = r"""
You are a systems engineer tasked with analyzing text to identify variables and their causal relationships for constructing a Causal Loop Diagram (CLD). 

1. **Identify Primary Variables**  
   - First, read through the original input text carefully.  
   - Extract all key entities or components (e.g., “salesmen,” “orders,” “product delivery,” “revenue,” etc.).  
   - If certain variables overlap or share commonalities, group them thematically (e.g., “sales workforce,” “budget,” “capacity,” etc.).  

2. **Break Down Sub-Factors**  
   - Within each primary variable, look for nuanced factors that might influence or be influenced by others (e.g., “delivery delay,” “acquisition delay,” “order backlog”).  
---

### Example of How to Apply This Prompt

Example 1:
**Input Excerpt**  
```
"More chickens lay more eggs, which hatch and add to the chicken population.
The more chickens, the more road crossing they will attempt. If there is any
traffic, more road crossings will lead to fewer chickens."
```

**Output**  
### Step 1: Identify Primary Variables  
The key entities or components in the text are:  

1. **Chickens**  
2. **Eggs**  
3. **Road Crossings**  
4. **Traffic**  

---

### Step 2: Break Down Sub-Factors  

For each primary variable, here are the nuanced sub-factors:  

1. **Chickens**  
   - Chicken population size  
   - Chickens attempting to cross roads  

2. **Eggs**  
   - Number of eggs laid  
   - Rate of eggs hatching  

3. **Road Crossings**  
   - Frequency of road crossing attempts by chickens  
   - Success rate of road crossings  

4. **Traffic**  
   - Presence of traffic  
   - Severity or density of traffic impacting road crossings  

Let me know if you'd like further refinement of these factors or additional steps for mapping the causal relationships.

Example 2:

**Input Excerpt**  
```
"Here is a situation in which salesmen book orders followed by
product delivery which generates revenue which produces the sales
budget which permits hiring still more salesmen. In short, salesmen
produce revenue to pay for the further expansion of the sales
effort. Orders booked increase the order backlog which increases the
delivery delay which makes the product less attractive and reduces
the order rate. The ordering of new production capacity is a
function of delivery delay only. Rising order backlog, as indicated
by delivery delay, is taken as an indication of inadequate capacity,
and orders for more capacity are placed. These orders, after an
acquisition delay, add to the production capacity. As the delivery
delay rises, production capacity is raised to bring down the
delivery delay."
```

**Output**  
### Step 1: Identify Primary Variables  
The key entities or components in the text are:  

1. **Salesmen**  
2. **Orders Booked**  
3. **Order Backlog**  
4. **Delivery Delay**  
5. **Product Attractiveness**  
6. **Order Rate**  
7. **Revenue**  
8. **Sales Budget**  
9. **Production Capacity**  
10. **Acquisition Delay**  

---

### Step 2: Break Down Sub-Factors  

For each primary variable, here are the nuanced sub-factors:  

1. **Salesmen**  
   - Number of salesmen  
   - Sales effort exerted  

2. **Orders Booked**  
   - Total number of orders booked  
   - Rate at which orders are booked  

3. **Order Backlog**  
   - Accumulation of unfulfilled orders  
   - Rate of backlog clearance  

4. **Delivery Delay**  
   - Time taken to fulfill orders  
   - Dependency on production capacity  

5. **Product Attractiveness**  
   - Perceived value of the product  
   - Sensitivity to delivery delay  

6. **Order Rate**  
   - Frequency of new orders placed  
   - Dependency on product attractiveness  

7. **Revenue**  
   - Income generated from delivered orders  
   - Dependency on order fulfillment  

8. **Sales Budget**  
   - Allocation of funds for sales operations  
   - Dependency on revenue  

9. **Production Capacity**  
   - Total capacity to produce and fulfill orders  
   - Dependency on acquisition of new capacity  

10. **Acquisition Delay**  
   - Time taken to procure and integrate new production capacity  
   - Dependency on capacity orders  
"""

In [54]:
case = r"""
    Patient Information

    Patient Name: Mr. Smith
    Patient ID: 789456
    Date of Birth: 06/25/1965
    Date of Examination: 03/15/2024

MRI Examination Details

    MRI Technique: Multi-parametric MRI with T2-weighted images, diffusion-weighted imaging (DWI), and dynamic contrast-enhanced (DCE) MRI.
    Prostate Volume: 45 cc
    PSA Level: 5.8 ng/mL

MRI Findings

    Lesion 1: Located in the left peripheral zone, mid gland. It shows significant restricted diffusion with a corresponding low ADC value and focal early enhancement on DCE MRI. PI-RADS Score: 5.
    Lesion 2: Located in the right peripheral zone, apex. It has mildly restricted diffusion and no significant enhancement on DCE MRI. PI-RADS Score: 3.
    Lesion 3: Located in the anterior fibromuscular stroma, showing moderate restricted diffusion and no early enhancement. PI-RADS Score: 3."""

In [55]:
messages = [
    {"role": "system", "content": system_message_1},
    {"role": "user", "content": case},
]

In [56]:
# Send the messages to the OpenAI API and receive a response
first_response = client.chat.completions.create(
    model="chatgpt-4o-latest",
    messages=messages,
    temperature=0.1 # Set the temperature for response creativity (lower is more deterministic)
)

# Extract the assistant's reply
first_reply = first_response.choices[0].message.content
print(first_reply)

### Step 1: Identify Primary Variables  
The key entities or components in the text are:  

1. **Patient Information**  
   - Patient Name  
   - Patient ID  
   - Date of Birth  
   - Date of Examination  

2. **MRI Technique**  
   - Multi-parametric MRI  
   - T2-weighted images  
   - Diffusion-weighted imaging (DWI)  
   - Dynamic contrast-enhanced (DCE) MRI  

3. **Prostate Volume**  
   - Measured in cubic centimeters (cc)  

4. **PSA Level**  
   - Measured in nanograms per milliliter (ng/mL)  

5. **Lesions**  
   - Lesion 1: Left peripheral zone, mid gland  
   - Lesion 2: Right peripheral zone, apex  
   - Lesion 3: Anterior fibromuscular stroma  

6. **PI-RADS Score**  
   - Lesion 1: PI-RADS 5  
   - Lesion 2: PI-RADS 3  
   - Lesion 3: PI-RADS 3  

---

### Step 2: Break Down Sub-Factors  

For each primary variable, here are the nuanced sub-factors:  

1. **Patient Information**  
   - Patient demographics (e.g., name, ID, date of birth)  
   - Examination date  

2. **M

In [57]:
system_message_2 = r"""
You are a systems engineer tasked with analyzing text to identify variables and their causal relationships for constructing a Causal Loop Diagram (CLD). Follow these steps:

3. **Map Causal Relationships**  
   - For each pair of related variables, specify the relationship in the following format:  
     \[
       [Variable A] --> ([+/-]) [Variable B]
     \]  
     Use **(+)** if an increase in Variable A leads to an increase in Variable B. Use **(-)** if an increase in Variable A leads to a decrease in Variable B.  
   - Provide your **Reasoning** next: Explain how you determined the relationship. If something in the original text supports or illustrates this cause-effect linkage, reference it.  
   - Under **Relevant Text**, include the **exact sentence(s) or phrase(s)** from the original input text that justifies your relationship. Enclose these quotes clearly (e.g., in quotation marks).


---

**Output Format**

For each relationship, follow this exact template:

1. **[Variable A] --> ([+/-]) [Variable B]**  
   **Reasoning:** [Explanation of how/why the relationship is positive or negative, referencing delays and any loops if applicable.]  
   **Relevant Text:** [Exact quote(s) from the original text that substantiate this relationship.]

---

### Example of How to Apply This Prompt

Example 1:
**Input Excerpt**  
```
"More chickens lay more eggs, which hatch and add to the chicken population.
The more chickens, the more road crossing they will attempt. If there is any
traffic, more road crossings will lead to fewer chickens."
```

**Output**  
1. chickens --> (+) eggs  
   Reasoning: Increased chickens lead to more eggs being laid.  
   Relevant Text: "More chickens lay more eggs..."

2. eggs --> (+) chickens  
   Reasoning: Eggs hatch into chickens, thus increasing the chicken population.  
   Relevant Text: "...which hatch and add to the chicken population."

3. road crossing --> (-) chickens  
   Reasoning: More chickens crossing roads, combined with traffic, can lead to a decreased chicken population.  
   Relevant Text: "If there is any traffic, more road crossings will lead to fewer chickens."

Example 2:

**Input Excerpt**  
```
"Here is a situation in which salesmen book orders followed by
product delivery which generates revenue which produces the sales
budget which permits hiring still more salesmen. In short, salesmen
produce revenue to pay for the further expansion of the sales
effort. Orders booked increase the order backlog which increases the
delivery delay which makes the product less attractive and reduces
the order rate. The ordering of new production capacity is a
function of delivery delay only. Rising order backlog, as indicated
by delivery delay, is taken as an indication of inadequate capacity,
and orders for more capacity are placed. These orders, after an
acquisition delay, add to the production capacity. As the delivery
delay rises, production capacity is raised to bring down the
delivery delay."
```

**Output**  
1. Salesmen --> (+) Orders booked
Reasoning: Salesmen are responsible for booking orders which lead to
revenue generation.
Relevant Text: Here is a situation in which salesmen book orders
followed by product delivery which generates revenue which produces the
sales budget which permits hiring still more salesmen.
2. Revenue --> (+) Sales budget
Reasoning: Revenue generated from orders booked contributes to the
sales budget.
Relevant Text: Here is a situation in which salesmen book orders
followed by product delivery which generates revenue which produces the
sales budget which permits hiring still more salesmen.
3. Sales budget --> (+) Salesmen
Reasoning: The sales budget allows for the hiring of more salesmen,
expanding the sales effort.
Relevant Text: Here is a situation in which salesmen book orders
followed by product delivery which generates revenue which produces the
sales budget which permits hiring still more salesmen.
4. Orders booked --> (+) Order backlog
Reasoning: Orders booked contribute to the order backlog.
Relevant Text: Orders booked increase the order backlog which increases
the delivery delay which makes the product less attractive and reduces
the order rate.
5. Order backlog --> (+) Delivery delay
Reasoning: An increase in order backlog leads to longer delivery
delays.
Relevant Text: As the delivery delay rises, production capacity is
raised to bring down the delivery delay.
6. Delivery delay --> (-) Product attractiveness
Reasoning: Longer delivery delays make the product less attractive to
customers.
Relevant Text: Orders booked increase the order backlog which increases
the delivery delay which makes the product less attractive and reduces
the order rate.
7. Product attractiveness --> (-) Order rate
Reasoning: Reduced product attractiveness leads to a decrease in the
order rate.
Relevant Text: Orders booked increase the order backlog which increases
the delivery delay which makes the product less attractive and reduces
the order rate.
8. Delivery delay --> (+) Capacity orders
Reasoning: The decision to order new production capacity is based on
the current delivery delay.
Relevant Text: The ordering of new production capacity is a function of
delivery delay only.
9. Delivery delay --> (+) Capacity orders
Reasoning: Delivery delay is used as an indicator of inadequate
capacity, prompting orders for more capacity.
Relevant Text: Rising order backlog, as indicated by delivery delay, is
taken as an indication of inadequate capacity, and orders for more
capacity are placed.
10. Capacity orders --> (+) Production capacity
Reasoning: After an acquisition delay, the ordered capacity adds to the
existing production capacity.
Relevant Text: These orders, after an acquisition delay, add to the
production capacity.
11. Delivery delay --> (+) Production capacity
Reasoning: As delivery delay increases, production capacity is raised
to reduce the delivery delay.
Relevant Text: As the delivery delay rises, production capacity is
raised to bring down the delivery delay.
12. Production capacity --> (-) Delivery delay
Reasoning: Increased production capacity should reduce the delivery
delay.
Relevant Text: As the delivery delay rises, production capacity is
raised to bring down the delivery delay.
13. Order rate --> (-) Orders booked
Reasoning: Decreased order rate leads to fewer orders booked.
Relevant Text: Orders booked increase the order backlog which increases
the delivery delay which makes the product less attractive and reduces
the order rate.
14. Orders booked --> (-) Revenue
Reasoning: Fewer orders booked would lead to a decrease in revenue.
Relevant Text: Here is a situation in which salesmen book orders
followed by product delivery which generates revenue which produces the
sales budget which permits hiring still more salesmen.
"""

In [60]:
messages = [
    {"role": "system", "content": system_message_2},
    {"role": "user", "content": case + first_reply},
]

In [61]:
# Send the messages to the OpenAI API and receive a response
second_response = client.chat.completions.create(
    model="chatgpt-4o-latest",
    messages=messages,
    temperature=0.1 # Set the temperature for response creativity (lower is more deterministic)
)

# Extract the assistant's reply
second_reply = second_response.choices[0].message.content
print(second_reply)

### Step 3: Map Causal Relationships  

Below, I will identify causal relationships between the variables and explain the reasoning behind each relationship based on the provided text.

---

1. **Prostate Volume --> (+) PSA Level**  
   **Reasoning:** An increase in prostate volume is often associated with a higher PSA level, as a larger prostate produces more prostate-specific antigen.  
   **Relevant Text:** "Prostate Volume: 45 cc" and "PSA Level: 5.8 ng/mL" (While not explicitly stated, this is a well-established medical relationship.)  

---

2. **Lesion Characteristics (e.g., restricted diffusion, ADC value, enhancement) --> (+) PI-RADS Score**  
   **Reasoning:** Lesions with significant restricted diffusion, low ADC values, and focal early enhancement are more likely to receive a higher PI-RADS score, indicating a higher likelihood of clinically significant prostate cancer. For example, Lesion 1 has these characteristics and is assigned a PI-RADS score of 5.  
   **Relevant Tex

In [63]:
system_message_3 = r"""
You are a systems engineer tasked with analyzing text to identify variables and their causal relationships for constructing a Causal Loop Diagram (CLD). Follow these steps:

4. **Add Feedback Loops**  
   - Identify any cyclical cause-effect paths (e.g., “salesmen → revenue → budget → more salesmen”).  
   - Label loops as either *Reinforcing (R)* or *Balancing (B)*, depending on whether they amplify or stabilize the system.  
   - Include a short note on how the loop works overall, mentioning how the sign of relationships causes growth (reinforcing) or equilibrium (balancing).  

5. **Account for Delays**  
   - If the text mentions time delays in processes (e.g., “acquisition delay,” “delivery delay”), highlight them.  
   - Explain how delays might shift or weaken the speed of the feedback loop’s response.  

6. **Suggest Interventions**  
   - Based on the relationships identified, propose one or more policy or managerial interventions that could alter the system’s outcome (e.g., “shorten acquisition delay,” “increase capacity earlier”).  

---

**Output Format**

For each relationship, follow this exact template:

- A consolidated **Feedback Loops** section, listing and labeling each loop (e.g., Loop R1, Loop B1) and a brief explanation of how it reinforces or balances the system.  
- A brief note on **Delays**, describing their impact on the system.  
- **Suggestions** or recommended interventions.  

---

### Example of How to Apply This Prompt

Example 1:
**Input Excerpt**  
```
"More chickens lay more eggs, which hatch and add to the chicken population.
The more chickens, the more road crossing they will attempt. If there is any
traffic, more road crossings will lead to fewer chickens."
```

**Output**  
Feedback Loops

    Loop R1 (Reinforcing):
    Path: chickens → (+) eggs → (+) chickens
    Explanation: This reinforcing loop demonstrates growth in the chicken population as more chickens lay eggs, which hatch and further increase the population.

    Loop B1 (Balancing):
    Path: chickens → (+) road crossing → (-) chickens
    Explanation: This balancing loop acts to stabilize or decrease the chicken population. As the number of chickens grows, road crossings increase, and traffic reduces the chicken population.

Delays

There may be a time delay between:

    Eggs being laid and chickens hatching. This delay slows the impact of the reinforcing loop (R1).
    Chickens crossing roads and the population being reduced due to traffic. This could introduce a lag in the balancing loop (B1).

Suggestions

    Minimize Road Crossing: Implement barriers or safe crossing zones to reduce traffic-related chicken losses.
    Manage Population Growth: Introduce measures to control the chicken population (e.g., regulating egg production) to avoid overpopulation and excessive road crossings.

Example 2:

**Input Excerpt**  
```
"Here is a situation in which salesmen book orders followed by
product delivery which generates revenue which produces the sales
budget which permits hiring still more salesmen. In short, salesmen
produce revenue to pay for the further expansion of the sales
effort. Orders booked increase the order backlog which increases the
delivery delay which makes the product less attractive and reduces
the order rate. The ordering of new production capacity is a
function of delivery delay only. Rising order backlog, as indicated
by delivery delay, is taken as an indication of inadequate capacity,
and orders for more capacity are placed. These orders, after an
acquisition delay, add to the production capacity. As the delivery
delay rises, production capacity is raised to bring down the
delivery delay."
```

**Output**  
Feedback Loops

    Loop R1 (Reinforcing): Sales Effort Expansion
    Path: salesmen → (+) orders booked → (+) revenue → (+) sales budget → (+) salesmen
    Explanation: This reinforcing loop shows how sales revenue funds the hiring of more salesmen, leading to further growth in sales.

    Loop B1 (Balancing): Delivery Delay Impact on Orders
    Path: orders booked → (+) order backlog → (+) delivery delay → (-) order rate → (-) orders booked
    Explanation: This balancing loop highlights how rising delivery delays reduce the order rate, limiting further growth in orders.

    Loop B2 (Balancing): Capacity Adjustment
    Path: delivery delay → (+) production capacity orders → (+) production capacity (after delay) → (-) delivery delay
    Explanation: This balancing loop regulates delivery delay by adjusting production capacity.

Delays

    Acquisition Delay: There is a delay between placing orders for new production capacity and when it becomes operational.
    Delivery Delay: Time is required to clear the backlog of orders, impacting customer satisfaction and order rates.

Suggestions

    Improve Delivery Times: Invest in production process optimization to reduce delivery delays and retain customer interest.
    Forecast Capacity Needs: Use predictive analytics to anticipate production needs and order capacity proactively, reducing dependency on reactionary measures.
    Balance Sales Growth with Delivery Efficiency: Ensure that sales growth does not outpace production capacity to avoid excessive delivery delays.
"""

In [64]:
messages = [
    {"role": "system", "content": system_message_3},
    {"role": "user", "content": case + first_reply + second_reply},
]

In [65]:
# Send the messages to the OpenAI API and receive a response
third_response = client.chat.completions.create(
    model="chatgpt-4o-latest",
    messages=messages,
    temperature=0.1 # Set the temperature for response creativity (lower is more deterministic)
)

# Extract the assistant's reply
third_reply = third_response.choices[0].message.content
print(third_reply)

### Step 4: Add Feedback Loops  

Based on the identified relationships, here are the feedback loops present in the system:

---

**Feedback Loops**

1. **Loop R1 (Reinforcing): Diagnostic Escalation**  
   **Path:** PSA Level → (+) Likelihood of Prostate Cancer Investigation → (+) Detection of Lesion Characteristics → (+) PI-RADS Score → (+) Likelihood of Clinically Significant Prostate Cancer → (+) PSA Level  
   **Explanation:** Elevated PSA levels prompt further diagnostic investigations (e.g., MRI), which detect lesion characteristics and assign PI-RADS scores. Higher PI-RADS scores increase the likelihood of clinically significant prostate cancer, which in turn reinforces the need for PSA monitoring and further diagnostic escalation.  

2. **Loop B1 (Balancing): Prostate Volume and PSA Specificity**  
   **Path:** Prostate Volume → (+) PSA Level → (-) Accuracy of PSA Level as a Cancer Indicator → (-) Likelihood of Prostate Cancer Investigation  
   **Explanation:** Larger prostat

In [66]:
print(case + first_reply + second_reply + third_reply)


    Patient Information

    Patient Name: Mr. Smith
    Patient ID: 789456
    Date of Birth: 06/25/1965
    Date of Examination: 03/15/2024

MRI Examination Details

    MRI Technique: Multi-parametric MRI with T2-weighted images, diffusion-weighted imaging (DWI), and dynamic contrast-enhanced (DCE) MRI.
    Prostate Volume: 45 cc
    PSA Level: 5.8 ng/mL

MRI Findings

    Lesion 1: Located in the left peripheral zone, mid gland. It shows significant restricted diffusion with a corresponding low ADC value and focal early enhancement on DCE MRI. PI-RADS Score: 5.
    Lesion 2: Located in the right peripheral zone, apex. It has mildly restricted diffusion and no significant enhancement on DCE MRI. PI-RADS Score: 3.
    Lesion 3: Located in the anterior fibromuscular stroma, showing moderate restricted diffusion and no early enhancement. PI-RADS Score: 3.### Step 1: Identify Primary Variables  
The key entities or components in the text are:  

1. **Patient Information**  
   - Patien

In [67]:
messages = [
    {"role": "system", "content": "You are a design thinking expert. You will review the input to identify if there are any missing information or unrealised cause-effect in the whole chain. Revise it, and then visualise the CLD in ASCII format. "},
    {"role": "user", "content": case + first_reply + second_reply + third_reply},
]

In [68]:
# Send the messages to the OpenAI API and receive a response
response = client.chat.completions.create(
    model="chatgpt-4o-latest",
    messages=messages,
    temperature=0.1 # Set the temperature for response creativity (lower is more deterministic)
)

# Extract the assistant's reply
final_reply = response.choices[0].message.content
print(final_reply)

### Causal Loop Diagram (CLD) in ASCII Format

Below is the ASCII representation of the causal loop diagram (CLD) based on the relationships, feedback loops, and delays identified in the analysis:

```
                          +----------------+
                          |   Prostate     |
                          |   Volume (cc)  |
                          +----------------+
                                   |
                                   | (+)
                                   v
                          +----------------+
                          |    PSA Level   |
                          |  (ng/mL)       |
                          +----------------+
                                   |
                                   | (+)
                                   v
                          +----------------+
                          | Likelihood of  |
                          | Prostate Cancer|
                          | Investigation  |
                          +-